In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

import logging

import verde as vd
import xarray as xr
import pandas as pd
import numpy as np
from polartoolkit import maps
from polartoolkit import utils as polar_utils
import string

from invert4geom import inversion, plotting, synthetic, utils

In [2]:
# set grid parameters
spacing = 1e3
inversion_region = [0, 80e3, 0, 60e3]

buffer_region = vd.pad_region(inversion_region, pad=6e3)
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

# create synthetic topography data
true_topography = synthetic.synthetic_topography_simple(
    spacing,
    buffer_region,
)

inversion region =  [0, 80000.0, 0, 60000.0]
buffer region =  (-6000.0, 86000.0, -6000.0, 66000.0)


In [3]:
num_constraints = 30
coords = vd.scatter_points(
    region=vd.pad_region(inversion_region, -2e3), size=num_constraints, random_state=1
)
constraint_points = pd.DataFrame(data={"easting": coords[0], "northing": coords[1]})

x_range = abs(inversion_region[0] - inversion_region[1]) / 1000
y_range = abs(inversion_region[2] - inversion_region[3]) / 1000
inversion_area = x_range * y_range

const_density = inversion_area / num_constraints

print(f"inversion region: {inversion_area} km\u00b2")
print(f"constraint density: 1 constraint per {const_density} km\u00b2")
print(f"Ross Ice Shelf constraint density: 1 constraint per {487000 / 224} km\u00b2")

# sample true topography at these points
constraint_points = utils.sample_grids(
    constraint_points, true_topography, "upward", coord_names=("easting", "northing")
)
constraint_points.head()

inversion region: 4800.0 km²
constraint density: 1 constraint per 160.0 km²
Ross Ice Shelf constraint density: 1 constraint per 2174.1071428571427 km²


,easting,northing,upward
0,33693.672357,7507.422695,469.360206
1,56744.661502,25582.027000,515.234600
2,2008.692486,55641.813688,652.349330
3,24977.275520,31857.255958,467.788477
4,13153.447702,40745.118381,411.819148


In [4]:
# grid the sampled values using verde
starting_topography = utils.create_topography(
    method="splines",
    region=buffer_region,
    spacing=spacing,
    constraints_df=constraint_points,
    dampings=np.logspace(-20, 0, 100),
)

In [5]:
# the density contrast is between rock (~2670 kg/m3) and air (~1 kg/m3)
true_density_contrast = 2670 - 1

# prisms are created between the mean topography value and the height of the topography
true_zref = true_topography.values.mean()

# prisms above zref have positive density contrast and prisms below zref have negative
# density contrast
density_grid = xr.where(
    true_topography >= true_zref, true_density_contrast, -true_density_contrast)

# create layer of prisms
prisms = utils.grids_to_prisms(
    true_topography,
    true_zref,
    density=density_grid,
)

In [6]:
# make pandas dataframe of locations to calculate gravity
# this represents the station locations of a gravity survey
# create lists of coordinates
coords = vd.grid_coordinates(
    region=inversion_region,
    spacing=spacing,
    pixel_register=False,
    extra_coords=1000,  # survey elevation
)

# grid the coordinates
observations = vd.make_xarray_grid(
    (coords[0], coords[1]),
    data=coords[2],
    data_names="upward",
    dims=("northing", "easting"),
).upward

grav_df = vd.grid_to_table(observations)

grav_df["observed_grav"] = prisms.prism_layer.gravity(
    coordinates=(
        grav_df.easting,
        grav_df.northing,
        grav_df.upward,
    ),
    field="g_z",
    progressbar=True,
)

grav_df

  0%|          | 0/4941 [00:00<?, ?it/s]

,northing,easting,upward,observed_grav
0,0.0,0.0,1000.0,11.830044
1,0.0,1000.0,1000.0,11.410575
2,0.0,2000.0,1000.0,10.983736
3,0.0,3000.0,1000.0,10.553015
4,0.0,4000.0,1000.0,10.120588
...,...,...,...,...
4936,60000.0,76000.0,1000.0,2.079946
4937,60000.0,77000.0,1000.0,2.075750
4938,60000.0,78000.0,1000.0,2.077695
4939,60000.0,79000.0,1000.0,2.085408


# Zref-Density Cross Validation

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

# set kwargs to pass to the inversion
kwargs = {
    "starting_prisms": starting_prisms,
    "grav_data_column": "observed_grav",
    "deriv_type": "annulus",
    # set stopping criteria
    "max_iterations": 100,
    "l2_norm_tolerance": 0.05,
    "delta_l2_norm_tolerance": 1.02,
    # for creating test/train splits
    "grav_spacing":spacing,
    "inversion_region":inversion_region,
    # for calculating the regional component of misfit
    "regional_grav_kwargs": {
        "regional_method": "constant",
        "constant": 0,
    },
}

# run the inversion workflow, including a cross validation for the damping parameter
results = inversion.run_inversion_workflow(
    grav_df=grav_df,
    calculate_gravity_misfit=True,
    calculate_regional_misfit=True,
    run_damping_cv=True,
    damping_values=np.logspace(-3, 0, 10),
    plot_cv=True,
    cv_progressbar=True,
    **kwargs,
)

# collect the results
topo_results, grav_results, parameters, elapsed_time = results

solver_damping values:   0%|          | 0/10 [00:00<?, ?it/s]